This notebook, adapted from https://github.com/Ranlot/public-key-encryption.

Please visit the original notebook for further details on padding, signature, optimizations, etc.

---

<center><a href="https://colab.research.google.com/github/Textualization/riiaa21_ws11_ml_over_encrypted_data/blob/main/notebooks/1_RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></center>



In [1]:
!pip install egcd

## 0) Some essential background


### Converting from base 10 to other bases

We only need to be able to convert from base from base 10 to base 2.

*(Incidentally, the ``converter`` below will also convert to other bases &#8804; 10)*


In [2]:
def eulidean_division(x, y):
    return x // y, x % y

def verif(a, base):
    return sum([int(val) * base ** int(pos) for pos, val in enumerate(str(a)[::-1])])

# positive integers and base <= 10 only
# (we don't include special symbols for higher bases)
def converter(a, base):
    q, r = eulidean_division(a, base)
    xs = [str(r), ]
    while q > 0:
        q, r = eulidean_division(q, base)
        xs.append(str(r))
    result = int(''.join(xs[::-1]))
    assert verif(result, base) == a
    return result

In [3]:
print(converter(194759, 2))

101111100011000111


### Euclid's extended algorithm

Instead of implementing Euclid's extended algorithm ourselves, we decide to rely on the `egcd` library for functionality and focus on a few interesting outcomes of the theorem:
- Bézout's identity
- Modular inverses

In [4]:
from egcd import egcd 

def euclidAlg(fst, sec):
    # egcd would not be difficult to implement but we focus on the resulting properties:
    # a) Bézout's identity
    # b) Modular inverses
    
    gcd, fst_inv, sec_inv = egcd(fst, sec)
        
    # a) Bézout's identity
    assert gcd == (fst * fst_inv) + (sec * sec_inv)
    
    # b) Modular inverses
    assert gcd == fst * fst_inv % sec
    assert gcd == sec * sec_inv % fst
    
    return {'gcd' : gcd, 'fst_inv' : fst_inv % sec, 'sec_inv' : sec_inv % fst}

In case that the input numbers $(a, b)$ are coprimes (a very common situation in this notebook), their greatest common divisor is $\text{gcd}(a, b) = 1$ and we have the following identities:

\begin{equation}
a \cdot a^{-1}_{\%b} + b \cdot b^{-1}_{\%a} = 1 \quad
\text{where} \quad  
\begin{pmatrix}
a \cdot a^{-1}_{\%b} \, \equiv 1 \,\,\text{mod}\,\,b \\
b \cdot b^{-1}_{\%a} \, \equiv 1 \,\,\text{mod}\,\,a
\end{pmatrix}
\end{equation}

Note that $\text{gcd}(a,b)$ and the modular inverses $a^{-1}_{\%b}$ and $b^{-1}_{\%a}$ are all computed efficiently and simultaneously with `egcd`. 

### Fast modular exponentiation

We are following a classic *square and multiply* method.  

(Numbers never exceed the modulus and the complexity only grows linearly with the number of bits of the modulus.)

In [5]:
def multiplyList(xs, m) :
    result = 1
    for x in xs:
        result = result * x % m
    return result % m

def FasterModularExponentiation(b, e, m):
    def helper(kmax):
        d = {}
        s = b % m
        d[0] = s
        for _ in range(kmax):
            s = (s * s) % m
            d[_ + 1] = s
        return d

    e2 = converter(e, 2)
    necessaryBits = [_[0] for _ in enumerate(reversed(str(e2))) if _[1] == '1']
    d = helper(max(necessaryBits))
    allVals = [d[_] for _ in necessaryBits]

    return multiplyList(allVals, m)

In [6]:
FasterModularExponentiation(14, 9, 138)

44

Quick check with small integers

In [7]:
14 ** 9 % 138

44

# 1) Real-world SSH keys

We start by using **ssh-keygen** to generate a pair of public / private keys into directory _ssh_key_example_.  By default the private key **ssh_key** (default name when generating the key) is in PEM (Privacy-Enhanced Mail) format but the public key is not.  Given the private key, one can generate a PEM public key by running:

```openssl rsa -in ssh_key -pubout -out ssh_key.pub.pem```

This way the contents of both keys can be parsed with openssl.

## 1.1) Public Key

The public key is composed of 2 integers:

---
- the modulus denoted as $n$,
- and the public exponent $e$
---

Both are represented below in hexadecimal representation.


In [8]:
!rm -f /content/key
!ssh-keygen -t rsa -f /content/key -P ""

Generating public/private rsa key pair.
Your identification has been saved in /content/key.
Your public key has been saved in /content/key.pub.
The key fingerprint is:
SHA256:0gh6Pmvi2pFRYspzMC6vGXFT3b64YjrzDJZntc93K4s root@34623e0a3edd
The key's randomart image is:
+---[RSA 2048]----+
|                 |
|     . .         |
| oo + . .        |
|ooo= . +         |
|++=.. + S        |
|.+oB . + .       |
|. B = o .        |
| *+*+o + .o .    |
|+o+O+.. E..+..   |
+----[SHA256]-----+


In [9]:
!openssl rsa -in /content/key -pubout -out /content/key.pub.pem

writing RSA key


In [10]:
!openssl asn1parse -inform pem -in /content/key.pub.pem > /content/pub.parsed.txt
!cat /content/pub.parsed.txt
parse1 = open("/content/pub.parsed.txt").read().split("\n")

    0:d=0  hl=4 l= 290 cons: SEQUENCE          
    4:d=1  hl=2 l=  13 cons: SEQUENCE          
    6:d=2  hl=2 l=   9 prim: OBJECT            :rsaEncryption
   17:d=2  hl=2 l=   0 prim: NULL              
   19:d=1  hl=4 l= 271 prim: BIT STRING        


In [11]:
pos = list(map(lambda x:"BIT STRING" in x,parse1)).index(True)
pos = int(parse1[pos].split(':')[0])
pos

19

In [12]:
!openssl asn1parse -inform pem -in /content/key.pub.pem -strparse 19 > /content/pub.parsed2.txt
!cat /content/pub.parsed2.txt
parse2 = open("/content/pub.parsed2.txt").read().split("\n")

    0:d=0  hl=4 l= 266 cons: SEQUENCE          
    4:d=1  hl=4 l= 257 prim: INTEGER           :AC05FFB7102213DC13A9014E8C4C703918EF623DEF82846479E0347A9A7EB6A363BFE81ADD1D74A03E76F902FDBCA939509CC3ECA36FE1AD2FCEA9B7B60BE6CA17C1D724923D2E7E1A4A0B4AFA20E228BC31F5C1321BD20264173BAC8CBDA50096B202007FD6AA36B1F2E74BE501B10530E7250BDE0F571FABEA0A0B8C570319BA649759F018B8580FA0C3357D54254C3B176EE3CA1AA8A0069F9E856570896023C0618D9B7DF27EB6F3E1833E2F53B54943428AF357E53EC3C3D465BB4355B3F55273F086087E81E59F4462B26AE3880DDA53E4D6EB7B2A5D4C8DD117FE3535DE5C5EBD020A19EF5E9E176195C7810517C8AF7080DE6DB105972D451CDFE3D7
  265:d=1  hl=2 l=   3 prim: INTEGER           :010001


In [13]:
pem_public_n=int(parse2[1].split("prim: INTEGER")[1].split(":")[1], 16)

In [14]:
print('public modulus =', pem_public_n)

pem_public_e = int('010001', 16)

print('\npublic exponent =', pem_public_e)

public modulus = 21715946615628491221009512461320695768691930240655262567642852775269874524952838801307460819361241199497284042094032492568400727118059350118809392263393249193312246209889263901269288602246028518954198572754811716659463454672169496714083685227354347329973296586052841448153403869846816025106020217229912034421687048309643280215864953850002189120360940423581286340390442257193874461861266089925757001227755993890918134849037904375930990506725962705012312684286066780950953908462774308020999399973254693148261831369634571958185545210380221985747142484611094910664415384197008231021017205087647922542776238167216976028631

public exponent = 65537


By convention, the public exponent is fixed to a 65537.  This value is rather small when compared to the modulus and the notebook will have a section to below to discuss the order of magnitudes of the different numbers.  We will check later (when we have access to the private key) that this value is indeed coprime to $\phi(n)$ as should be for RSA to work)

## 1.2) Private key

Obviously, the private key contains more information which is composed of the following 5 integers:

---
*(repeated from public key)*
- the modulus $n$
- the public exponent $e$
---
- the private exponent $d$ used for decrytion
- the first random prime $p$
- the second random prime $q$
---
*(to be used for optimized decryption via Chinese remainder theorem)*
- the first optizmized exponent $d_p$
- the second optizmized exponent $d_q$
- modular inverse $q^{-1}_{\%p}$


In [15]:
!openssl asn1parse -inform pem -in /content/key  > /content/parsed.txt
!cat /content/parsed.txt
parse3 = open("/content/parsed.txt").read().split("\n")

    0:d=0  hl=4 l=1189 cons: SEQUENCE          
    4:d=1  hl=2 l=   1 prim: INTEGER           :00
    7:d=1  hl=4 l= 257 prim: INTEGER           :AC05FFB7102213DC13A9014E8C4C703918EF623DEF82846479E0347A9A7EB6A363BFE81ADD1D74A03E76F902FDBCA939509CC3ECA36FE1AD2FCEA9B7B60BE6CA17C1D724923D2E7E1A4A0B4AFA20E228BC31F5C1321BD20264173BAC8CBDA50096B202007FD6AA36B1F2E74BE501B10530E7250BDE0F571FABEA0A0B8C570319BA649759F018B8580FA0C3357D54254C3B176EE3CA1AA8A0069F9E856570896023C0618D9B7DF27EB6F3E1833E2F53B54943428AF357E53EC3C3D465BB4355B3F55273F086087E81E59F4462B26AE3880DDA53E4D6EB7B2A5D4C8DD117FE3535DE5C5EBD020A19EF5E9E176195C7810517C8AF7080DE6DB105972D451CDFE3D7
  268:d=1  hl=2 l=   3 prim: INTEGER           :010001
  273:d=1  hl=4 l= 257 prim: INTEGER           :832BABDAB65595D929B0A44B75D5CF78EA57970CED3613A7DDFB25691BA765B2DF1BF56B8E91A85D8C6401EAD2FA69FB4749D267FE1410FC9348BDC754EC6C564B9946691F8DC186EC9AEB5387B94D5A8C6E781B920EFEAB4E111D32ACBAEB37B0B119AFB1CB494D9D913FE7723F40F15961D7B0DDE9C

In [16]:
def get_num(idx):
  return int(parse3[idx].split("prim: INTEGER")[1].split(":")[1], 16)

We start by verifying that $n$ are $e$ are indeed identical between the public and private keys:

In [17]:
pem_private_n = get_num(2)

pem_private_e = int('010001', 16)

In [18]:
assert pem_private_n == pem_public_n
assert pem_private_e == pem_public_e

The following three consist of the private exponent $d$ and the two underlying primes $p, q$ that divide the factorize the (public) modulus

In [19]:
pem_private_d = get_num(4)

pem_private_p = get_num(5)

pem_private_q = get_num(6)

print('p =', pem_private_p)
print('\nq =', pem_private_q)
print('\nd =', pem_private_d)

p = 160038932688010088722468117741559201514057148941636814865407349974194912683371030331089472288639400841032028049966386630027212959171134582891014430468358627532986382306617730015951038412994350432613310018087860139388241653554724639583880623489757337226547046708135617931412786417113140237742295346612561066933

q = 135691648593801338496124726421389725381418198695874442886431013880758336178782551214060895482570994207158621895232615059797657058361105813036976369463753708111954813559883070358857908757688718198859425068521773904309523571127061836524451089257560862421221619697907612589475660713626725774221462586450358079707

d = 165587530741840882522469500622484875665477795736189547323316032430315919196098114563946738411269863811660249238684267780203654353459996628391177771270969199374001385453529484861701048159061413122007135705979249266311147477048434664280162961665407143921869409691444320870435029309833366986987983629954131879112591104389796249412549338319545348457825302690104562644719

We can check that the modulus $n = pq$ is indeed the product of the private random primes.  In addition, we also check that the public exponent $e$ is coprime to Euler's totient function $\varphi(n)$. Since $p$ and $q$ are primes, we have $$\varphi(n) = \varphi(p)\cdot\varphi(q) = (p-1)\cdot(q-1)$$.

In [20]:
assert pem_public_n == pem_private_p * pem_private_q
assert 1 == euclidAlg(pem_private_e, (pem_private_p - 1) * (pem_private_q - 1))['gcd']

Therefore we can verify that the private exponent is indeed the inverse of the public exponent modulo $\varphi(n)$

\begin{equation}
e \cdot d \equiv 1 \,\, \text{mod} \,\, \varphi(n)
\end{equation}

In [21]:
assert 1 == pem_public_e * pem_private_d % ((pem_private_p - 1) * (pem_private_q - 1))

The other numbers contained in the private key are not strictly necessary for decryption.  They will come in handy when using the Chinese remainder theorem to optimize the decryption part (discussed further below in the notebook).

In [22]:
pem_crt_exponent_p = get_num(7)

pem_crt_exponent_q = get_num(8)

pem_crt_qInv = get_num(9)

In [23]:
from math import log10, log, floor

def approx10(x): # take a power an exponent in base 2 and approximate in base 10
    b10exp = floor(log10(x))
    prefactor = round(x / 10 ** b10exp, 3)
    return '%s x 10^{%s}' % (prefactor, b10exp)

## 3.2) Encryption

The cipher is determined by the remainder of the division of the plaintext $m$ raised to the public exponent $e$ by the public modulus $n$

\begin{equation}
c = m^e \,\,\text{mod}\,\, n
\end{equation}

We use the square and multiply modular exponentation function implemented earlier in the notebook

In [24]:
def encrypt(msgInt):
  return FasterModularExponentiation(msgInt, pem_public_e, pem_public_n)

encrypted = encrypt(42)
encrypted

8190426316951588871705024152037450723259184493868029297644553600410982488480687435175286875855091785133848843322930728180168916619668151111547153409115560011136420993534596343159441057226211741903871094581272843149476972764367163674852849883562432505938689823203896680462805225938911380775160514773190901640403024394868135040259562079226225802879793344446474129388751968495931444496868505660548040843911005313961219866191066606528847851697173445993685438390464414267585026983206269649148345123944823959556601439505017228902651580852900462240680442380234265883739870613156255736847699501930738727870957806061321179841


## 3.3) Decryption

Similar to the encryption, we will show first how to decode the cipher using the Pycrypto library and then verify that our custom modular exponentiation function reproduces the same result.  The message can be recovered by evaluating the remainder of the division of the cipher $c$ raise to the private exponent $d$ with the public modulus $n$:

\begin{equation}
c^d \,\,\text{mod}\,\, n \equiv m \,\,\text{mod}\,\,n
\end{equation}

The proof of why this is true is given in the accompanying PDF document.  Note that the private exponent $d$ is very large (on the order to the modulus $n$) since it is defined as the modular inverse of the public exponent $e$ which is itself quite small:

\begin{equation}
e \cdot d \equiv 1 \,\,\text{mod}\,\, \phi(n)
\end{equation}


#### The private exponent $d$

One of the strenghts of RSA is that it is not feasible to evaluate Euler's totient function $\phi(n)$ without having previous knowledge of $p$ and $q$.  In other words, calculating $\phi(n)$ is as hard as factorizing $n$ into its prime factors.  However, if we do know the values of $p, q$ it is very easy to calculate $\phi(n)$ as

\begin{align}
\phi(n) &= \phi(p) \cdot \phi(q) \\
&= (p-1) \cdot (q-1)
\end{align}

In [25]:
phiN = (pem_private_p - 1) * (pem_private_q - 1)

First of all, we shoud verify that $e$ and $\phi(n)$ are co-primes (actually it was already checked earlier but we can do it again...).  This is a necessary condition for the modular inverse of $e$ to exist (and be unique).

In [26]:
egcd_e_phiN = euclidAlg(pem_public_e, phiN)

assert egcd_e_phiN['gcd'] == 1

In this case, $d$ is also calculated using Euclid's extended theorem.

In [27]:
my_private_d = egcd_e_phiN['fst_inv']

print('e = %d' % pem_public_e)
print('\nd = ', my_private_d)
print('\nd ~ %s' % (approx10(my_private_d)))
print('\nd / n ~ %s' % approx10(my_private_d / pem_public_n))

e = 65537

d =  16558753074184088252246950062248487566547779573618954732331603243031591919609811456394673841126986381166024923868426778020365435345999662839117777127096919937400138545352948486170104815906141312200713570597924926631114747704843466428016296166540714392186940969144432087043502930983336698698798362995413187911259110438979624941254933831954534845782530269010456264471961676027677644341435362944658060236567941940699461420259418098673743979835480491201471192255064551250093432512528434282971077374657243877161016045159302720611224639902771224242493411391252476915862587975767794833345048161810371378218290045949321521641

d ~ 1.656 x 10^{616}

d / n ~ 7.625 x 10^{-1}


With $e = 65537$, we have a private exponent $d \approx 8.5 \cdot 10^{615} \approx 0.35 \, n$. Because this decryption exponent is so big, it is more efficient to optimize the decryption using the Chinese remainder theorem than to perform a straightforward exponentiation directly on $c$.  

We will show in a section below how to do this optimization, but for now in the sake of simplicity, let us go ahead and calculate $c^d \,\,\text{mod}\,\,n$ directly.

In [28]:
def decrypt(my_cipher):
  return FasterModularExponentiation(my_cipher, my_private_d, pem_public_n)

decrypted = decrypt(encrypted)
decrypted

42

Verify that the initial message is indeed recovered by all methods

In [29]:
assert 42 == decrypted

### Sanity check

As a sanity check, let us change the private exponent $d$ into a fake $d_\text{fake} = d - 1$ almost identical to the real one.  Notice that we are talking about a difference of 1 to numbers on the order of $10^{615}$ and yet, as expected, the decryption completely fails to recover the original message.

In [30]:
FasterModularExponentiation(encrypted, my_private_d - 1, pem_public_n)

9072473313442528896626598910250559962116229715338150829343351273789281330311576958671482180886926560447076708926452579595787856408595403828903755035645514371686478339042677622177480207661767397439816016168050647234598146899608822472289180759956241976386976326267109020128109276790350036394836636533728011047048278733594438695330813768713920086315689677833818870974148525692906056798179422856719577551014994238154391314457255619108441579104485170305400014888024815209056351321566987259849777744218606988733321750218294364091857068783667458858807347920931844133941467915100895140231771397352783421640378931953471157597

# 5) Partial homomorphic encryption in "textbook" RSA

In this section, we show how unpadded RSA is a partially homomorphic encryption scheme.  

Let us consider two integers $x_1$ and $x_2$ on which we wish to perform some computation.  Instead of doing the computation locally, we want to perform it on some unsecure data processor (cloud for example...) without revealing to the remote server anything about our data other than the list of instructions to execute.


Denoting by $\mathcal{E}$ the "textbook" RSA encryption function, the secret numbers $x_1$ and $x_2$ are encrypted into ciphers $c_1$ and $c_2$

\begin{align}
c_1 &= \mathcal{E}(x_1) \\
c_2 &= \mathcal{E}(x_2) 
\end{align}

In [31]:
x1 = 585
x2 = 117

In [32]:
c1 = encrypt(x1)
c2 = encrypt(x2)

In [33]:
print('x1 =', x1)
print('c1 =',hex(c1))

print('\nx2 =', x2)
print('c2 =', hex(c2))

x1 = 585
c1 = 0x280e718a78828d2fc9514ed998fbbe50da9ad10a999c881a9c07e4bf805f6ae3f994ce3a8efc92923de3a6d45a537d06b0912e435b74fc401aa220c5d681d72bac3140cf166ab8eff39966b280abc4db45dac366d9d4f30f5495f08a1411ef6cc11af4844a67cd53f6dffb1c505d3c0ef1e51d862d9b28a23029aceed8c9b15f448bd06066070523213d19036ebbae25142e3a1b33036f94cac78e78e66309d5ee19b220abccff63e26601cb29ebebce4ba17996b71fd63a54160ddfbc17f4fba8aa3db0a26c70036635a2409e95355e434f3b69cfdb8ec45e838538d8206e6fed00afddfd998c7dc054c219bb61de538d88539467875075784cdb0e42baa1

x2 = 117
c2 = 0x2adbed6afb3d3b34e148e70dcacaafa1379306e110bd51f0e5ec24dcf4bb11079dcb28243a0ec6d32078baedaa0e5dbfe5d46c33f6b4a583b9175a2b0da9976fc0adb218b138bbdb7eb29262214c9c3f0b4d97a0758cd0508c0ca09fb58bc3060fc613ac93de9fceaa76dbd17d08fc7837cffa98038531eb84c47440489df10372b956e636b2d514670c8b104287ee98a6b8e2044dda5014f62db1ab8d8805160403c510c6b1d660c9ad5983dd8e1718774ce38884b4baa0a598dfe34c66b493c8996743d264572da15d329093cb5622548cadcd46ff1e51f9316b17b311e4250f8e18ae

The encrypted ciphers $c_2$ and $c_2$ can now be sent to an untrusted environment tasked with excecuting arithmetic instructions on the ciphers.

## 5.1) Multiplication

In [34]:
c1Mulc2 = c1 * c2

print('c1Mulc2 = ', hex(c1Mulc2))

c1Mulc2 =  0x6b4c821f5a275c6ce2d0e19ac851d25d4262889c9fd8d1c49b9261284c83cc6a1cb57b5e6945f268c169bc778b3f54b79a55d6262b1e8b860f6f3622a44c18ec289b3afbdeaa3cba38ba4c026c2dc0da61298f585486a7c4ee1873f56d846839a3df056bbc110972b36ec7332f8159aee44d58b50472bfcc3a3ca6430708f01857b499186064e7879b6cbd41541b13641d4dfd98e58fac584f530a5082a1e68c053a651baf97d2332b14cd7dc4e413c182919fdec088c8f60dc2a0b3579223267e5822c27d74db974f904d53856e1aed4e776a5fa6658fd583c5dcb636171dd1576cd4011ba53477eeefa004e08851d451a274ba5e2a9f6f8bc77f0761f6570ba31ed5454ec00d2d08ff54e0e65a0d91322c51afc2dd9c58abc75a14db07ee4191096ba970ff84d5e9e6bfd89911c8099a98c607861d47505006d568c40d82bd281562360f2c4bf1fdb09c7011bb11bd00390da08251f6418ea65f603270edb90e9e6a28e8277faac8c3f118ef07a24a3b8564b512ae2032962df84eade310a11eacf5e2f250b02465e3d78372adc5709f3258788bf0746a2dfc27824efcc89d63f92ccbcd8096a3e98badd3efe33ef9ea19123c7bada3cededce96a122261a7bd99e3d42ddba2d87d5b85794cc1204d2a6d702101caf1491ef42d44516efb114578ee2cd75ef8956ec34db08b1

Send the result of $c_1 \cdot c_2$ back to the data owner who is the only one able to decrypt

In [35]:
decrypt(c1Mulc2)

68445

which turns out to be exactly $x_1 \cdot x_2$ even though these numbers were never communicated to anyone

In [36]:
print(x1 * x2)

68445


**Why does this happen?**

For a more in-depth derivation, we refer to the accomapnying PDF document.  In a nutshell, it is easy to show that RSA encryption, denoted as $\mathcal{E}$ satisfies the multiplicative property: 

$$c_1 \cdot c_2 = \mathcal{E}(x_1) \cdot \mathcal{E}(x_2) = \mathcal{E}(x_1 \cdot x_2)$$

so that the decryption function $\mathcal{D}$ applied to the product $c_1\cdot c_2$ of the ciphers yields the product $x_1 \cdot x_2$ of the original two numbers.

$$\mathcal{D} (c_1 \cdot c_2) = \mathcal{D} (\mathcal{E}(x_1 \cdot x_2)) = x_1 \cdot x_2 $$

Notice that the untrusted environment never learned anything about $x_1, x_2$ or even about the result of $x_1 \cdot x_2$ since all it ever was exposed to were the encrypted ciphers $c_1, c_2$ and $c_1 \cdot c_2$.  Nevertheless, thanks to multiplicatice homomorphicity of RSA, it was able to perform a useful computation.

## 5.3) Addition / subtraction

Unfortunately, these operations are not homomorphic under RSA and one needs to resort to different encryption schemes...

In [37]:
decrypt(c1 + c2)

14146764356220182575751589768617352070529994166518578352199532999300738418264655227932311990157388947860973793544007829723498018651106304759976624570294689550856070445676517244900050907662209409819671096211975723156244816124885337325089989886616018427653519705677093766905619293698616872037022597416632065856689930434710011031466248365894560594216454732438292048158620392514100725686598565969263334875589572777986489031469697888657915272170433305507289077239318140145050197406405118048566081084496076949316153131486070036528306896987528800481786075800004816249507139041864021580656208659657959113332993255824833038728

which is clearly very different from

In [38]:
x1 + x2

702